<h1><center>ERM with DNN under penalty of Equalized Odds</center></h1>

We implement here a regular Empirical Risk Minimization (ERM) of a Deep Neural Network (DNN) penalized to enforce an Equalized Odds constraint. More formally, given a dataset of size $n$ consisting of context features $x$, target $y$ and a sensitive information $a$ to protect, we want to solve
$$
\text{argmin}_{h\in\mathcal{H}}\frac{1}{n}\sum_{i=1}^n \ell(y_i, h(x_i)) + \lambda \chi^2|_1
$$
where $\ell$ is for instance the MSE and the penalty is
$$
\chi^2|_1 = \left\lVert\chi^2\left(\hat{\pi}(h(x)|y, a|y), \hat{\pi}(h(x)|y)\otimes\hat{\pi}(a|y)\right)\right\rVert_1
$$
where $\hat{\pi}$ denotes the empirical density estimated through a Gaussian KDE.

### Imports

In [1]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../..')))
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as data_utils
import numpy as np

from examples.data_loading import read_dataset

### The dataset

We use here the _communities and crimes_ dataset that can be found on the UCI Machine Learning Repository (http://archive.ics.uci.edu/ml/datasets/communities+and+crime). Non-predictive information, such as city name, state... have been removed and the file is at the arff format for ease of loading.

In [2]:
sys.path.append(os.path.abspath(os.path.join('../..')))

In [3]:
x_train, y_train, a_train, x_test, y_test, a_test = read_dataset(name='crimes', fold=1)
n, d = x_train.shape

### The Deep Neural Network

We define a very simple DNN for regression here

In [4]:
class NetRegression(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NetRegression, self).__init__()
        size = 50
        self.first = nn.Linear(input_size, size)
        self.fc = nn.Linear(size, size)
        self.last = nn.Linear(size, num_classes)

    def forward(self, x):
        out = F.selu(self.first(x))
        out = F.selu(self.fc(out))
        out = self.last(out)
        out = torch.sigmoid(out) # NEW
        return out

### The fairness-penalized ERM

We now implement the full learning loop. The regression loss used is the quadratic loss with a L2 regularization and the fairness-inducing penalty.

In [14]:
def regularized_learning(x_train, y_train, a_train, model, fairness_metric_train, fairness_metric_test, fairness_weight = 1.0, lr=1e-5, num_epochs=10, print_progress = True):    
    X = torch.tensor(x_train.astype(np.float32))
    A = torch.tensor(a_train.astype(np.float32))
    Y = torch.tensor(y_train.astype(np.float32))
    dataset = data_utils.TensorDataset(X, Y, A)
    dataset_loader = data_utils.DataLoader(dataset=dataset, batch_size=200, shuffle=True)

    # mse regression objective
    data_fitting_loss = nn.MSELoss()

    # stochastic optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.01)

    for j in range(num_epochs):
        if print_progress:
            print(f"EPOCH {j} started")
        for i, (x, y, a) in enumerate(dataset_loader):
            # if print_progress:
            #    print(f"Batch {i} started")
            def closure():
                optimizer.zero_grad()
                prediction = model(x).flatten()
                loss = fairness_weight * fairness_metric_train(prediction, a, y) + data_fitting_loss(prediction, y)
                loss.backward()
                """for name, param in model.named_parameters():
                    if param.grad is not None:
                        print(f"Parameter: {name}\nGradient: {param.grad}\n")"""
                return loss
            optimizer.step(closure)
        mse_curr, nd_curr = evaluate(model, x_test, y_test, a_test, fairness_metric=fairness_metric_test)
        print(f"mse: {mse_curr}, nd: {nd_curr}, combined: {mse_curr + fairness_weight * nd_curr}")
    return model

### Evaluation

For the evaluation on the test set, we compute two metrics: the MSE (accuracy) and HGR$|_\infty$ (fairness).

In [15]:
def evaluate(model, x, y, a, fairness_metric):
    X = torch.tensor(x.astype(np.float32))
    A = torch.Tensor(a.astype(np.float32))
    Y = torch.tensor(y.astype(np.float32))

    prediction = model(X).detach().flatten()
    loss = nn.MSELoss()(prediction, Y)
    discrimination = fairness_metric(prediction, A, Y)
    return loss.item(), discrimination

### Running everything together


In [16]:
def generate_fairness_metric(constrained_intervals_A, quantizition_intervals_Y, train, size_compensation = lambda x: np.sqrt(x)):
    
    def inside(num, endpoints):
        start, end = endpoints
        return start <= num and num < end
    
    def fairness_metric(Y_hat, A, Y):
        nd_losses = []
        n = len(Y_hat)
        for inter_Y in quantizition_intervals_Y:
            for inter_A in constrained_intervals_A:
                cnt_y_a = 0
                cnt_y = 0
                sum_y_yhat = 0
                sum_y_a_yhat = 0
                for i in range(len(Y_hat)): # could be sped up by combining with outer loop
                    if inside(Y[i], inter_Y):
                        cnt_y += 1
                        sum_y_yhat += Y_hat[i]
                        if inside(A[i], inter_A):
                            cnt_y_a += 1
                            sum_y_a_yhat += Y_hat[i]
                # print(inter_Y, inter_A, cnt_y, cnt_y_a)
                if cnt_y_a > 0 and cnt_y > 0:
                    curr_nd_loss = torch.abs(sum_y_a_yhat / cnt_y_a - sum_y_yhat / cnt_y) * size_compensation(cnt_y_a / n)
                    nd_losses.append(curr_nd_loss)
        nd_losses_torch = torch.stack(nd_losses)
        return torch.mean(nd_losses_torch) if train else torch.max(nd_losses_torch)
    return fairness_metric

In [17]:
def generate_constrained_intervals(num_constrained_intervals):
    endpoints = np.linspace(0, 1, num_constrained_intervals + 1)
    constrained_intervals = []
    for i in range(len(endpoints) - 1):
        constrained_intervals.append((endpoints[i], endpoints[i + 1]))
    return constrained_intervals

In [19]:
model = NetRegression(d, 1)
num_epochs = 100
lr = 1e-5
fairness_weight = 10
num_constrained_intervals = 2
intervals = generate_constrained_intervals(num_constrained_intervals)
fairness_metric_train = generate_fairness_metric(intervals, intervals, True)
fairness_metric_test = generate_fairness_metric(intervals, intervals, False)

model = regularized_learning(x_train, y_train, a_train, model=model, fairness_metric_train=fairness_metric_train, fairness_metric_test=fairness_metric_test, lr=lr, \
                             num_epochs=num_epochs, fairness_weight=fairness_weight)
mse, discrimination = evaluate(model, x_test, y_test, a_test, fairness_metric=fairness_metric_test)

EPOCH 0 started
mse: 0.12971177697181702, nd: 0.005482755601406097, combined: 0.184539332985878
EPOCH 1 started
mse: 0.128629669547081, nd: 0.0047569251619279385, combined: 0.17619892954826355
EPOCH 2 started
mse: 0.1275639683008194, nd: 0.004030788317322731, combined: 0.1678718477487564
EPOCH 3 started
mse: 0.1265244483947754, nd: 0.0033072440419346094, combined: 0.15959689021110535
EPOCH 4 started
mse: 0.1255127489566803, nd: 0.0026167347095906734, combined: 0.1516800969839096
EPOCH 5 started
mse: 0.12463998794555664, nd: 0.001991408411413431, combined: 0.14455407857894897
EPOCH 6 started
mse: 0.12381522357463837, nd: 0.0014226344646885991, combined: 0.13804157078266144
EPOCH 7 started
mse: 0.12295719236135483, nd: 0.0008402536623179913, combined: 0.13135972619056702
EPOCH 8 started
mse: 0.12225601077079773, nd: 0.0009659646893851459, combined: 0.13191565871238708
EPOCH 9 started
mse: 0.12156471610069275, nd: 0.0010645337169989944, combined: 0.13221004605293274
EPOCH 10 started
mse: 

mse: 0.10780078172683716, nd: 0.001676041167229414, combined: 0.12456119060516357
EPOCH 84 started
mse: 0.10744552314281464, nd: 0.0018897948320955038, combined: 0.1263434737920761
EPOCH 85 started
mse: 0.10725725442171097, nd: 0.0019270391203463078, combined: 0.12652763724327087
EPOCH 86 started
mse: 0.10714548826217651, nd: 0.0018847991013899446, combined: 0.12599347531795502
EPOCH 87 started
mse: 0.10700961947441101, nd: 0.0019347110064700246, combined: 0.12635673582553864
EPOCH 88 started
mse: 0.10692167282104492, nd: 0.0019421711331233382, combined: 0.12634338438510895
EPOCH 89 started
mse: 0.10682743042707443, nd: 0.0019600125961005688, combined: 0.126427561044693
EPOCH 90 started
mse: 0.10683954507112503, nd: 0.0018397268140688539, combined: 0.12523680925369263
EPOCH 91 started
mse: 0.10662498325109482, nd: 0.0019439999014139175, combined: 0.1260649859905243
EPOCH 92 started
mse: 0.1064852699637413, nd: 0.001967216143384576, combined: 0.12615743279457092
EPOCH 93 started
mse: 0.

In [11]:
model = NetRegression(d, 1)
num_epochs = 100
lr = 1e-5
fairness_weight = 10
num_constrained_intervals = 2
intervals = generate_constrained_intervals(num_constrained_intervals)
fairness_metric_train = generate_fairness_metric(intervals, intervals, True)
fairness_metric_test = generate_fairness_metric(intervals, intervals, False)

model = regularized_learning(x_train, y_train, a_train, model=model, fairness_metric_train=fairness_metric_train, fairness_metric_test=fairness_metric_test, lr=lr, \
                             num_epochs=num_epochs, fairness_weight=fairness_weight)
mse, discrimination = evaluate(model, x_test, y_test, a_test, fairness_metric=fairness_metric_test)

In [10]:
print("MSE:{} Beta loss:{}".format(mse, discrimination))

MSE:0.13390439748764038 Beta loss:0.005223618820309639


In [12]:
q

NameError: name 'q' is not defined

In [ ]:
c = a + 4 * b
d = 5 * a + b
l = [c, d]
t = torch.tensor(l)
q = torch.mean(t)
q.backward()

In [ ]:
print(t)
print(r)
print(q)

In [ ]:
baba = torch.tensor([1.0, 2.0], requires_grad=True)
kaka = torch.tensor([3.0, 4.0], requires_grad=True)
t = torch.sum(baba*kaka)
r = torch.sum(baba / kaka)
q = torch.mean(torch.tensor([t, r], requires_grad = True))
q.backward()

In [ ]:
baba._grad, kaka._grad

In [ ]:
a = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
c = a + 4 * b
d = 5 * a + b

# Set requires_grad=True for c and d
c.retain_grad()
d.retain_grad()

l = [c, d]
t = torch.tensor(l)
q = torch.mean(t)
q.backward()

In [ ]:
for param in model.parameters():
    print(param._grad)

In [ ]:
# Create a tensor with requires_grad=True
t = torch.tensor([1.0, 2.0, 3.0], requires_grad=True)

# Perform seven differentiable calculations on t
calculation1 = t * 2
calculation2 = t ** 2
calculation3 = torch.sin(t)
calculation4 = t / 2
calculation5 = torch.exp(t)
calculation6 = torch.cos(t)
calculation7 = t + 1

# Store the results in a list
results = [calculation1, calculation2, calculation3, calculation4, calculation5, calculation6, calculation7]

# Compute the mean of the results
mean_result = torch.mean(torch.stack(results))

# Perform the backward pass on the mean
mean_result.backward()

# Access the gradient of t
gradient_wrt_t = t.grad

# Print the gradient with respect to t
print("Gradient with respect to t:", gradient_wrt_t)

In [ ]:
torch.tensor(torch.stack(results))

In [ ]:
torch.stack(results)

In [ ]:
results